In [1]:
import numpy as np
import pandas as pd
import subprocess as sub
import matplotlib.pyplot as plt
import time

## step 1. Generate sample set by metropolis method

In [7]:
start = time.time()
sub.run(['g++', 'eff_step1_plaquette.cpp', '-o', 'step1_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step1_e'])
print("step1 finished :", time.time()-start)

#Data of Energy&nn in fileout.txt
f = pd.read_csv('fileout_eff.txt', sep = ' ', header = 0)
size = f.s[0]
temp = f.temp[0]
step2 = f.step2[0]
J0 = np.array(f.ene)
J1 = np.array(f.nn)
J2 = np.array(f.nnn)
J3 = np.array(f.nnnn)
num = len(J0)
nth = int(f.nth[0]) #How much consideration of nearest neighbors
Jmat = np.array([J1, J2, J3]).T
target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
print("T =", temp, " Energy, J =", Jlist, " error =", err)

#Data of coefficient in filein.txt
with open("filein_eff.txt", "w") as file:
    file.write('%d\n' %size)
    file.write('%d\n' %nth)
    file.write('%d\n' %step2)
    file.write('%.10g\n' %temp)
    for i in range (nth + 1):
        data = Jlist[i]
        file.write('%.10g\n' % data)

step1 finished : 316.93079233169556
T = 4.493  Energy, J = [2.5710452  1.07958708]  error = [15759.14218425]


## step 2. optimize J, regenerate sample set by wolff method

In [ ]:
sub.run(['g++', 'eff_step2_optimize.cpp', '-o', 'step2_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib','-ltrng4', '-std=gnu++11'])
for i in range (15):
    start = time.time()
    sub.run(['./step2_e'])
    print("step2 :", time.time()-start)

    #Data of Energy&nn in fileout.txt
    f = pd.read_csv('fileout_eff.txt', sep = ' ', header = 0)
    size = f.s[0]
    temp = f.temp[0]
    step2 = f.step2[0]
    J0 = np.array(f.ene)
    J1 = np.array(f.nn)
    J2 = np.array(f.nnn)
    J3 = np.array(f.nnnn)
    num = len(J0)
    nth = int(f.nth[0]) #How much consideration of nearest neighbors
    Jmat = np.array([J1, J2, J3]).T
    target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
    Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
    print("T =", temp, " Energy, J =", Jlist, " error =", err)
    #Data of coefficient in filein.txt
    with open("filein_eff.txt", "w") as file:
        file.write('%d\n' %size)
        file.write('%d\n' %nth)
        file.write('%d\n' %step2)
        if temp>2.5: file.write('%.10g\n' %temp)
        else: file.write('%.10g\n' %(temp+0.2))
        for i in range (nth + 1):
            data = Jlist[i]
            file.write('%.10g\n' % data)

step2 : 452.98674058914185
T = 4.293  Energy, J = [2.51146773 1.08022451]  error = [17013.88706387]
step2 : 401.63293194770813
T = 4.093  Energy, J = [3.86383308 1.08923423]  error = [15997.54019664]
step2 : 344.58879375457764
T = 3.8930000000000002  Energy, J = [3.38201189 1.0868451 ]  error = [14662.06623769]
